## Assignment 6

In [262]:
from typing import Mapping, Callable

In [217]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from numpy import linalg

sns.set()

np.random.seed(11148705)

### Parameters

In [136]:
sigmas = {
    'alpha': 1,
    'epsilon': 1,
    'xi': 1
}

coeff = {
    'rho': 0.5,
    'pi': 1,
    'theta': 0,
    'beta': 1
}

N = 100
T = 5

### Data generation

In [173]:
def stationary_x(alpha: np.array, N: int, rho:float, pi: float, theta:float, **_)->np.array: 
    
    den = 1-rho
    den_squared = np.sqrt(den * (1+rho))
    v = np.random.normal(0, 1, (N,1))
    xi = np.random.normal(0, 1, (N,1))
    
    alpha_init = (pi*alpha[:,0]) / den
    disturbance_init = (theta*v + xi) / den_squared
    
    
    return alpha_init + disturbance_init[:, 0]

In [179]:
def generate_data(
    T: int,
    N: int,
    sigmas: Mapping[str, float],
    coeff: Mapping[str, float]
) -> np.array:
    
    alpha_unstacked = np.random.normal(0, sigmas['alpha'], (N, 1))
    alpha = alpha_unstacked @ np.ones((1, T))
    
    epsilon = np.random.normal(0, sigmas['epsilon'], (N, T))
    
    init_x = stationary_x(alpha, N, **coeff)
    
    X = np.zeros((N, T))
    X[:, 0] = init_x
    
    xi = np.random.normal(0, sigmas['xi'], (N, T))
        
    for t in range(1, T):
        X[:, t] = coeff['rho'] * X[:, t-1] + coeff['pi'] * alpha[:, 0] + coeff['theta'] * epsilon[:, t-1]
        
    X = X + xi

    y = coeff['beta'] * X + alpha + epsilon
    
    
    return y, X, alpha

In [292]:
y, X, alpha = generate_data(T, N, sigmas, coeff)

### Estimators

In [296]:
def pooled(y, X, *rest):
    intercept = np.kron(np.identity(N), np.ones((T, 1)))
    W = np.concatenate((intercept, X.reshape(-1, 1)), axis=1)
        
    W_var = linalg.inv(W.T@W)
    
    theta_hat = W_var @ W.T @ y.reshape(-1, 1)     
    
    return theta_hat[-1, 0]

In [318]:
def fixed_effects(y, X, *rest):
    N, T = X.shape
    
    Q = np.identity(T) - np.ones((T, T))/T
    X_dem = (X @ Q).reshape(-1, 1)
    inv = linalg.inv(X_dem.T@X_dem)
    beta_hat = inv @ X_dem.T @ y.reshape(-1, 1)
    
    return beta_hat

In [319]:
fixed_effects(y, X)

array([[0.9418446]])

### Monte carlo

In [266]:
def monte_carlo_estimation(
    T, N, sigmas, coeff,
    estimators: Mapping[str, Callable],
    iterations: int = 1000
):
    
    for i in iterations:
        y, X, alpha = generate_data(T, N, sigmas, coeff)
        
        for method, estimate_fn in estimators.items():
            beta = estimate_fn(y, X, alpha)
        
    return
        

array([1.39530377])

In [201]:
X[0]

array([-1.46297987, -4.08633463, -2.06750154, -2.69886911, -4.13495333])

In [211]:
np.concatenate((np.ones((N*T, 1)), X.reshape(-1, 1)), axis=1)

array([[ 1.00000000e+00, -1.46297987e+00],
       [ 1.00000000e+00, -4.08633463e+00],
       [ 1.00000000e+00, -2.06750154e+00],
       [ 1.00000000e+00, -2.69886911e+00],
       [ 1.00000000e+00, -4.13495333e+00],
       [ 1.00000000e+00, -2.76240940e+00],
       [ 1.00000000e+00, -4.35048578e+00],
       [ 1.00000000e+00, -1.09535848e+00],
       [ 1.00000000e+00, -3.56934617e+00],
       [ 1.00000000e+00, -4.27512423e+00],
       [ 1.00000000e+00,  5.60960313e+00],
       [ 1.00000000e+00,  4.76099710e+00],
       [ 1.00000000e+00,  4.18389384e+00],
       [ 1.00000000e+00,  4.18507777e+00],
       [ 1.00000000e+00,  7.85904361e+00],
       [ 1.00000000e+00, -3.49244456e+00],
       [ 1.00000000e+00, -9.06205066e-01],
       [ 1.00000000e+00, -7.26859025e-01],
       [ 1.00000000e+00, -6.73063653e-01],
       [ 1.00000000e+00, -2.64798765e+00],
       [ 1.00000000e+00, -7.54678848e-01],
       [ 1.00000000e+00, -7.24515012e-03],
       [ 1.00000000e+00,  9.26157566e-02],
       [ 1.

In [208]:
 X.reshape(-1, 1).shape

(500, 1)

In [241]:
np.kron(np.identity(N), np.ones((T, 1))).shape

(500, 100)